In [66]:
from mpi4py import MPI

import numpy as np

import dolfinx.fem.petsc
import ufl

An example from https://jsdokken.com/fenics22-tutorial/helmholtz.html

In [67]:
import sys

from petsc4py import PETSc

if not np.issubdtype(PETSc.ScalarType, np.complexfloating):
    print("This tutorial requires complex number support")
    sys.exit(0)
else:
    print(f"Using {PETSc.ScalarType}.")

Using <class 'numpy.complex128'>.


In [68]:
# wavenumber in free space (air)
k0 = 10 * np.pi

# Corresponding wavelength
lmbda = 2 * np.pi / k0

# Polynomial degree
degree = 6

# Mesh order
mesh_order = 2

In [69]:
from dolfinx.io import gmshio
from mesh_generation import generate_mesh

# MPI communicator
comm = MPI.COMM_WORLD

file_name = "domain.msh"
generate_mesh(file_name, lmbda, order=mesh_order)
mesh, cell_tags, _ = gmshio.read_from_msh(file_name, comm, rank=0, gdim=2)

Info    : Reading 'domain.msh'...
Info    : 15 entities
Info    : 2985 nodes
Info    : 1444 elements
Info    : Done reading 'domain.msh'


In [70]:
W = dolfinx.fem.functionspace(mesh, ("DG", 0))
k = dolfinx.fem.Function(W)
k.x.array[:] = k0
k.x.array[cell_tags.find(1)] = 3 * k0

In [71]:
import matplotlib.pyplot as plt
import pyvista

from dolfinx.plot import vtk_mesh

#pyvista.start_xvfb()
pyvista.set_plot_theme("paraview")
sargs = dict(
    title_font_size=25,
    label_font_size=20,
    fmt="%.2e",
    color="black",
    position_x=0.1,
    position_y=0.8,
    width=0.8,
    height=0.1,
)


def export_function(grid, name, show_mesh=False, tessellate=False):
    grid.set_active_scalars(name)
    plotter = pyvista.Plotter(window_size=(700, 700))
    t_grid = grid.tessellate() if tessellate else grid
    plotter.add_mesh(t_grid, show_edges=False, scalar_bar_args=sargs)
    if show_mesh:
        V = dolfinx.fem.functionspace(mesh, ("Lagrange", 1))
        grid_mesh = pyvista.UnstructuredGrid(*vtk_mesh(V))
        plotter.add_mesh(grid_mesh, style="wireframe", line_width=0.1, color="k")
        plotter.view_xy()
    plotter.view_xy()
    plotter.camera.zoom(1.3)
    plotter.export_html(f"./{name}.html")

In [72]:
grid = pyvista.UnstructuredGrid(*vtk_mesh(mesh))
grid.cell_data["wavenumber"] = k.x.array.real

In [73]:
export_function(grid, "wavenumber", show_mesh=True, tessellate=True)

Next, is the step of defining the function of the boundary condition source term

In [74]:
n = ufl.FacetNormal(mesh)
x = ufl.SpatialCoordinate(mesh)
uinc = ufl.exp(1j * k * x[0])
g = ufl.dot(ufl.grad(uinc), n) - 1j * k * uinc

Then implement the weak form:

In [75]:
import basix.ufl

element = basix.ufl.element("Lagrange", mesh.topology.cell_name(), degree)
V = dolfinx.fem.functionspace(mesh, element)

u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

In [76]:
a = (
    -ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
    + k**2 * ufl.inner(u, v) * ufl.dx
    - 1j * k * ufl.inner(u, v) * ufl.ds
)
L = ufl.inner(g, v) * ufl.ds

Linear solver
Next, we will solve the problem using a direct solver (LU).

In [77]:
opt = {"ksp_type": "preonly", "pc_type": "lu"}
problem = dolfinx.fem.petsc.LinearProblem(a, L, petsc_options=opt)
uh = problem.solve()
uh.name = "u"

In [78]:
topology, cells, geometry = vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cells, geometry)
grid.point_data["Abs(u)"] = np.abs(uh.x.array)

abs_u_vals = np.abs(uh.x.array)

In [79]:
export_function(grid, "Abs(u)", show_mesh=False, tessellate=True)

In [80]:
from dolfinx.io import VTXWriter

u_abs = dolfinx.fem.Function(V, dtype=np.float64)
u_abs.x.array[:] = np.abs(uh.x.array)
# VTX can write higher order functions
with VTXWriter(comm, "out_high_order.bp", [u_abs], engine="BP4") as f:
    f.write(0.0)